In [1]:
import vaex
import pandas as pd
import panel as pn
import math
import param
import os
import glob
import datetime

In [2]:
pn.extension()

### Vaex

In [3]:
class Loader(param.Parameterized):
    
    file_path = param.String(default="", doc="Enter File Path")
    load_file = param.Action(lambda x: x.param.trigger('load_file'))
    
    
    def __init__(self, **params):
        super(Loader, self).__init__(**params)
        self.data = pd.DataFrame()
        self.data_length = len(self.data)
    
    @param.depends('load_file', watch=True)
    def load(self):
#         if self.file_path != "":
         self.data = vaex.open(str(self.file_path))
         self.data_length = len(self.data)
        

In [4]:
ldr = Loader()

In [5]:
ldr_test = pn.Row(ldr)
ldr_test

Row
    [0] Column(margin=5, name='Loader', width=300)
        [0] StaticText(value='<b>Loader</b>')
        [1] TextInput(name='File path')
        [2] Button(name='Load file')

In [7]:
class Frame(param.Parameterized):
    
    frame = param.DataFrame(pd.DataFrame())
    page = param.Integer(1)
    load = Loader()
    
    def __init__(self, **params):
        super(Frame, self).__init__(**params)
        self.per_page = 30

    @param.depends('load.load_file')
    def pagination(self):
        self.last_page = math.ceil(self.load.data_length / 30)
        self.param.page.bounds = (1, self.last_page)

    @param.depends('load.load_file', 'page')
    def refresh_frame(self):
        if self.load.file_path != "":
            self.from_item = ((self.page) - 1) * self.per_page
            self.to_item = (self.page * self.per_page)
            self.frame = self.load.data.take(range(self.from_item, self.to_item)).to_pandas_df()

    @param.depends('frame')
    def show_frame(self):
        df_widget = pn.widgets.DataFrame(self.frame, name= 'DataSlice', width=1200)
        return df_widget

In [8]:
fr_test = Frame()

In [9]:
pn.Row(pn.Column(fr_test.load, 
                 fr_test.param.page, 
                 fr_test.pagination,
                 ),
       fr_test.refresh_frame, 
       fr_test.show_frame
       )

Traceback (most recent call last):
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\pyviz_comms\__init__.py", line 316, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\panel\viewable.py", line 258, in _on_msg
 patch.apply_to_document(doc, comm.id)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in apply_to_document
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\document\document.py", line 1169, in _with_self_as_curdoc
 return f()
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 100, in <lambda>
 doc._with_self_as_curdoc(lambda: doc.apply_json_patch(self.content, setter))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\document\document.py", line 397, in apply_json_patch
 self._trigger_on_message(event_json["msg_type"], event_json["msg_data"])
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\document\document.py", line 686, in _trigger_on_message
 cb(msg_data)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\document\document.py", line 355, in apply_json_event
 model._trigger_event(event)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\util\callback_manager.py", line 85, in _trigger_event
 self._document._with_self_as_curdoc(invoke)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\document\document.py", line 1169, in _with_self_as_curdoc
 return f()
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\bokeh\util\callback_manager.py", line 74, in invoke
 callback(event)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\panel\widgets\button.py", line 103, in _server_click
 self._change_event(doc)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\panel\reactive.py", line 204, in _change_event
 self._process_events(events)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\panel\reactive.py", line 187, in _process_events
 self.param.set_param(**self._process_property_change(events))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 1451, in set_param
 self_._batch_call_watchers()
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 1578, in _batch_call_watchers
 watcher.fn(*events)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\panel\param.py", line 417, in action
 value(self.object)
 File "<ipython-input-3-3ec62126980f>", line 4, in <lambda>
 load_file = param.Action(lambda x: x.param.trigger('load_file'))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 1520, in trigger
 self_.set_param(**dict(params, **triggers))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 1451, in set_param
 self_._batch_call_watchers()
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 1578, in _batch_call_watchers
 watcher.fn(*events)
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 499, in caller
 return getattr(self,n)()
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\param\parameterized.py", line 337, in _depends
 return func(*args,**kw)
 File "<ipython-input-3-3ec62126980f>", line 15, in load
 self.data = vaex.open(str(self.file_path))
 File "C:\Users\Satya\.conda\envs\VaexPanel\lib\site-packages\vaex\__init__.py", line 182, in open
 raise IOError('Could not open file: {}, it does not exist'.format(path))
OSError: Could not open file: ./data/wheat_byAvMonth, it does not exist

Row
    [0] Column
        [0] Column(margin=5, name='Loader', width=300)
            [0] StaticText(value='<b>Loader</b>')
            [1] TextInput(name='File path')
            [2] Button(name='Load file')
        [1] IntInput(end=0, name='Page', start=1, value=1, value_throttled=1)
        [2] ParamMethod(method)
    [1] ParamMethod(method)
    [2] ParamMethod(method)

In [14]:
class Filter(param.parameterized):
    
    query = param.DataFrame()

SyntaxError: unexpected EOF while parsing (<ipython-input-14-c9edca28d6b5>, line 4)